In [3]:
import os
import dotenv
import boto3
import json

# AWS 계정 자격 증명 및 리전 설정
env_path = dotenv.find_dotenv()
dotenv.load_dotenv(env_path)

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
region_name = 'ap-northeast-2'

s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

def load_data(bucket_name, object_key):

    # JSON 파일 읽어오기
    try:
        response = s3.get_object(Bucket=bucket_name, Key=object_key)
        json_data = json.loads(response['Body'].read().decode('utf-8'))
        return json_data
    
    except Exception as e:
        print(e)

In [56]:
import numpy as np

json_data = load_data('mathcat-bucket', 'irt_result/yyyy=2024/mm=03/dd=15/irt_result.json')

result_array = []

for i in range(len(json_data['disc'])):
    new_array = [json_data['disc'][i], json_data['diff'][i], 0, 1]
    result_array.append(new_array)

np.array(result_array)

array([[ 0.99847758, -0.92409056,  0.        ,  1.        ],
       [ 1.11205029, -0.2648665 ,  0.        ,  1.        ],
       [ 0.91720927,  0.62256342,  0.        ,  1.        ],
       ...,
       [ 0.01898986,  0.06351005,  0.        ,  1.        ],
       [ 0.89896566, -0.49766356,  0.        ,  1.        ],
       [ 0.54160291, -0.06871212,  0.        ,  1.        ]])

In [68]:
# 초기 4문제 받는 부분

import numpy as np

# 난이도 추출
diff = np.array(result_array)[:, 1]

# 중앙값 계산
median_value = np.median(diff)

# 중앙값과 가장 가까운 네 개의 행의 인덱스를 찾기 위해 argsort 사용
# 두 번째 열의 값과 중앙값의 차이를 기준으로 오름차순 정렬 후 인덱스 반환
sorted_indices = np.argsort(np.abs(diff - median_value))

# 가장 작은 네 개의 인덱스 추출
selected_indices = sorted_indices[:4]

# 리스트 초기화
initial_item_ids = []

for index in selected_indices:
    item_id = json_data['item_ids'][str(index)]
    initial_item_ids.append(item_id)

initial_item_ids

['quiz30065553', 'quiz30066244', 'quiz30062306', 'quiz30044435']

In [5]:
from catsim.initialization import *
from catsim.selection import *
from catsim.estimation import *
from catsim.stopping import *

# 초기화
initializer = FixedPointInitializer(0)
selector = UrrySelector()
estimator = NumericalSearchEstimator()
stopper = MinErrorStopper(0.6)

In [87]:
from catsim.simulation import *
from catsim.irt import icc
import numpy as np

class Simulator:
    def __init__(self, result_array, max_items, initializer, selector, estimator, stopper):
        #simulator로 돌려야 되니까 self 로 그냥 다 받았습니다. 
        self.result_array = result_array
        self.max_items = max_items
        self.initializer = initializer
        self.selector = selector
        self.estimator = estimator
        self.stopper = stopper

    def init_est_ability(self, verbose=False):
        administered_items = []
        responses = []

        # 초기 4문제의 인덱스를 찾아서 administered_items에 추가하고 응답 벡터에 대응하는 응답을 추가
        for item_id in initial_item_ids:
            item_index = list(json_data['item_ids'].values()).index(item_id)
            administered_items.append(item_index)

            # 초기 문제에 대한 응답 추가
            correct = input(f"Did the user answer the item {item_id} correctly? (True/False): ").lower()
            if correct not in ['true', 'false']:
                print("Invalid input. Please enter either 'True' or 'False'.")
                continue
            responses.append(correct == 'true')

        # 초기화 단계
        est_theta = self.initializer.initialize()

        est_theta = self.estimator.estimate(
        items=np.array(self.result_array),
        administered_items=administered_items,
        response_vector=responses,
        est_theta=est_theta
    )
        # 여기까지가, 이제 해당 4문제들을 난이도가 중간값에 근사한 걸로 넣어가지고 진행했구요
        # 여기서 이제 사용자가 입력받아 True or False로 알겠지

    # 초기 4문제 후의 능력치 출력
        if verbose:
            print("Initial 4 items answered. Estimated proficiency:", est_theta)
            ## 여기까지 이제 4문제를 풀고 난후의 theta 값 즉, 사용자의 능력치를 추정

        if len(administered_items) == self.max_items:
            return est_theta, administered_items, responses
    
    def recommend_next(self, est_theta, administered_items, responses=None, verbose=False):

        if responses is None:
            pass # 5번째 문제부터는 응답을 받아서 값을 반환할 수 있도록 구현
        else:
            item_index = self.selector.select(items=np.array(self.result_array), administered_items=administered_items, est_theta=est_theta, last_response=responses[-1])
            next_item_id = json_data['item_ids'][str(item_index)]

        # 평가 단계
        true_theta = 0.0
        a, b, c, d = np.array(self.result_array)[item_index]
        prob = icc(true_theta, a, b, c, d)
        correct = input(f"Did the user answer the item {next_item_id} correctly? (True/False): ").lower()
        if correct not in ['true', 'false']:
            print("Invalid input. Please enter either 'True' or 'False'.")
            pass
        correct = (correct == 'true')

        # 결과 기록
        administered_items.append(item_index)
        responses.append(correct)

        # 능력치 업데이트
        est_theta = self.estimator.estimate(
            items=np.array(self.result_array),
            administered_items=administered_items,
            response_vector=responses,
            est_theta=est_theta
        )

        if verbose:
            print('Next item to be administered:', next_item_id)
            print('Probability to correctly answer item:', prob)
            print('Did the user answer the selected item correctly?', correct)
            print('Estimated proficiency: ', est_theta)

        return next_item_id, est_theta

In [89]:
# Simulator 객체 생성
simulator = Simulator(result_array, 4, initializer, selector, estimator, stopper)

# init_est_ability 메서드 호출
est_theta, administered_items, responses = simulator.init_est_ability(verbose=True)

# recommend_next 메서드 호출
next_item_id, est_theta = simulator.recommend_next(est_theta, administered_items, responses, verbose=True)

Initial 4 items answered. Estimated proficiency: 1.672370433807373
Next item to be administered: quiz30057096
Probability to correctly answer item: 0.08064003183031657
Did the user answer the selected item correctly? True
Estimated proficiency:  2.5085556507110596


In [23]:
from catsim.simulation import *
from catsim.irt import icc
import numpy as np

class Simulator:
    def __init__(self, result_array, max_items, initializer, selector, estimator, stopper):
        #simulator로 돌려야 되니까 self 로 그냥 다 받았습니다. 
        self.result_array = result_array
        self.max_items = max_items
        self.initializer = initializer
        self.selector = selector
        self.estimator = estimator
        self.stopper = stopper

    def simulate(self, verbose=False):
        administered_items = []
        responses = []

        # 초기 4문제의 인덱스를 찾아서 administered_items에 추가하고 응답 벡터에 대응하는 응답을 추가
        for item_id in initial_item_ids:
            item_index = list(json_data['item_ids'].values()).index(item_id)
            administered_items.append(item_index)

            # 초기 문제에 대한 응답 추가
            correct = input(f"Did the user answer the item {item_id} correctly? (True/False): ").lower()
            if correct not in ['true', 'false']:
                print("Invalid input. Please enter either 'True' or 'False'.")
                continue
            responses.append(correct == 'true')

        # 초기화 단계
        est_theta = self.initializer.initialize()

        est_theta = self.estimator.estimate(
        items=np.array(self.result_array),
        administered_items=administered_items,
        response_vector=responses,
        est_theta=est_theta
    )
        # 여기까지가, 이제 해당 4문제들을 난이도가 중간값에 근사한 걸로 넣어가지고 진행했구요
        # 여기서 이제 사용자가 입력받아 True or False로 알겠지

    # 초기 4문제 후의 능력치 출력
        if verbose:
            print("Initial 4 items answered. Estimated proficiency:", est_theta)
            ## 여기까지 이제 4문제를 풀고 난후의 theta 값 즉, 사용자의 능력치를 추정

        if len(administered_items) == self.max_items:
            return est_theta, administered_items, responses

        while True:
            # 추천 단계
            if len(administered_items) >= len(initial_item_ids):
                item_index = self.selector.select(items=np.array(self.result_array), administered_items=administered_items, est_theta=est_theta)
            else:
                # 초기 문제를 추천하지 않도록 건너뛰기
                item_index = administered_items[len(administered_items)]

            next_item_id = json_data['item_ids'][str(item_index)]
            ## 여기까지 이제 문제를 select해서 퀴즈코드 추천  -> 문제 추천해줘

            # 평가 단계
            true_theta = 0.0
            a, b, c, d = np.array(self.result_array)[item_index]
            prob = icc(true_theta, a, b, c, d)
            correct = input(f"Did the user answer the item {next_item_id} correctly? (True/False): ").lower()
            if correct not in ['true', 'false']:
                print("Invalid input. Please enter either 'True' or 'False'.")
                pass
            correct = (correct == 'true')

            # 여기까지가 그 문제를 맞추고, 틀렸는지 이제 estimation 해줘요

            # 중단 단계
            if len(administered_items) >= self.max_items or self.stopper.stop(administered_items=np.array(self.result_array)[administered_items], theta=est_theta):
                break
            #max_item 은 필요없긴 한데, self stopper에서 종단기준이 만족하면 stop

            # 결과 기록
            administered_items.append(item_index)
            responses.append(correct)

            if verbose:
                print('Estimated proficiency, given answered items:', est_theta)
                print('Next item to be administered:', next_item_id)
                print('Probability to correctly answer item:', prob)
                print('Did the user answer the selected item correctly?', correct)
                print("responses: ", responses)

            # 능력치 업데이트
            est_theta = self.estimator.estimate(
                items=np.array(self.result_array),
                administered_items=administered_items,
                response_vector=responses,
                est_theta=est_theta
            )

        if verbose:
            print("responses: ", responses)
        print("사용자의 최종 능력치 (Estimated Proficiency):", est_theta)
        # 사용자의 최종 능력치를 추정해줌.

In [24]:
s = Simulator(np.array(result_array), 4, initializer, selector, estimator, stopper)
s.simulate(verbose=True)

Initial 4 items answered. Estimated proficiency: 1.672370433807373


(1.672370433807373, [1131, 1252, 735, 822], [True, True, True, True])